In [259]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [260]:
df = pd.read_csv('melb_data.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


# Tampilkan statistik deskriptif pada variabel Rooms, Distance, Landsize, dan Price!

In [261]:
df.describe()[['Rooms', 'Distance', 'Landsize', 'Price']]

,Rooms,Distance,Landsize,Price
count,13580.000000,13580.000000,13580.000000,1.358000e+04
mean,2.937997,10.137776,558.416127,1.075684e+06
std,0.955748,5.868725,3990.669241,6.393107e+05
min,1.000000,0.000000,0.000000,8.500000e+04
25%,2.000000,6.100000,177.000000,6.500000e+05
50%,3.000000,9.200000,440.000000,9.030000e+05
75%,3.000000,13.000000,651.000000,1.330000e+06
max,10.000000,48.100000,433014.000000,9.000000e+06


#    

## a) Berapa jumlah Rooms pada rumah yang harganya paling mahal?

In [263]:
df.nlargest(1,'Price')[['Price', 'Rooms']]

,Price,Rooms
12094,9000000.0,3


## b) Berapa rata-rata harga rumah yang jarak ke CBD (variabel Distance) terdekat?

In [265]:
df.groupby('Distance').mean().sort_values('Distance')[['Price']].head(1)

,Price
Distance,
0.0,671166.666667


## c) Berapa rata-rata harga rumah yang luas tanahnya (variabel Landsize) 1 kali standar deviasi di atas rata-rata luas tanah?

In [27]:
std = df['Landsize'].std()
mean = df['Landsize'].mean()
df[ (df['Landsize'] >= mean-std) & (df['Landsize'] <= mean+std) ]['Price'].mean()

1076766.4975194372

## d) Berapa rentang (range) harga rumah (variabel Price) pada region Southern Metropolitan?

In [32]:
region_sm_price = df[ df['Regionname'] == 'Southern Metropolitan' ]['Price']
region_sm_price.max() - region_sm_price.min()

7869000.0

# Nilai outlier adalah nilai yang berada di bawah Q1 - (1.5 * IQR) atau di atas Q3 + (1.5 * IQR).

## a) Hitung dulu Q1, Q2, dan IQR pada kolom Price (harga rumah)

In [34]:
q1 = df['Price'].quantile(.25)
q2 = df['Price'].quantile(.5)
q3 = df['Price'].quantile(.75)

iqr = q3 - q1

print(f''' 
Q1 = {q1}
Q2 = {q2}
Q3 = {q3}
IQR = {iqr}
''')

 
Q1 = 650000.0
Q2 = 903000.0
Q3 = 1330000.0
IQR = 680000.0



## b) Hitung ada berapa nilai outlier pada kolom Price!

In [38]:
((df['Price'] < q1-1.5*iqr) | (df['Price'] > q3+1.5*iqr)).sum()

612

# Lakukan uji asumsi normalitas pada variabel harga rumah (Price) menggunakan

**Hypothesis**  
Ho = Harga rumah memiliki distribusi normal   
Ha = Harga rumah memiliki distribusi tidak normal

In [56]:
from scipy.stats import shapiro, normaltest

In [58]:
def normalitas(testresult):
    if testresult.pvalue > 0.05:
        print('Terima Ho, Harga rumah memiliki distribusi normal')
    else:
        print('Tolak Ho, Harga rumah memiliki distribusi tidak normal')

## a) Shapiro Wilk Test

In [59]:
normalitas(shapiro(df['Price']))

Tolak Ho, Harga rumah memiliki distribusi tidak normal


## b) D’Agostino and Pearson’s Test

In [60]:
normalitas(normaltest(df['Price']))

Tolak Ho, Harga rumah memiliki distribusi tidak normal


# Jika diasumsikan telah memenuhi syarat statistik parametrik, hitung seberapa kuat korelasi variabel Rooms dengan Price, serta arah korelasinya!

In [77]:
df['Rooms'].corr(df['Price'],method='pearson')

0.49663367618653054

nilainya 0.49 yang berarti korelasi antara room dan price sedang dan berbanding lurus

# 
## a) Dari beberapa Regionname, manakah Regionname yang variabel jumlah kamarnya (Rooms) memiliki korelasi terkuat dengan variabel harga rumah (Price)!
## b) Diasumsikan telah memenuhi syarat statistik parametrik.

In [176]:
# cara 1

# table correlation between rooms and price groupby Regionname
roomprice_corr_by_region = df[['Regionname', 'Rooms', 'Price']].groupby('Regionname').corr(method='pearson')

# make one level
roomprice_corr_by_region = roomprice_corr_by_region.unstack()

# find highest correlation pair
roomprice_corr_by_region[[('Rooms', 'Price')]].apply(abs).nlargest(1, ('Rooms', 'Price'))

,Rooms
,Price
Regionname,
Southern Metropolitan,0.683744


In [177]:
# cara 2 : one line code
df[['Regionname', 'Rooms', 'Price']].groupby('Regionname').corr(method='pearson').unstack()[[('Rooms', 'Price')]].apply(abs).nlargest(1, ('Rooms', 'Price'))


,Rooms
,Price
Regionname,
Southern Metropolitan,0.683744


# Jika diasumsikan tidak memenuhi syarat statistik parametrik, hitung seberapa kuat korelasi variabel Distance dengan Price, serta arah korelasinya!

In [148]:
df['Distance'].corr(df['Price'],method='spearman')

-0.12998984131880323

nilainya -0.12 yang berarti korelasi antara room dan price lemah dan berbanding terbalik

#  
## a) Dari beberapa Regionname manakah Regionname yang variabel jumlah jarak ke CBD(Distance) paling tidak berkorelasi dengan variabel harga rumah (Price)! 
## b) Diasumsikan telah memenuhi syarat statistik parametrik.

In [172]:
# cara 1

# table correlation between distance and price groupby Regionname
distanceprice_corr_by_region = df[['Regionname', 'Distance', 'Price']].groupby('Regionname').corr(method='pearson')

# make one level
distanceprice_corr_by_region = distanceprice_corr_by_region.unstack()

# find lowest correlation pair
distanceprice_corr_by_region[[('Price', 'Distance')]].apply(abs).sort_values(('Price', 'Distance')).head(1)

,Price
,Distance
Regionname,
Southern Metropolitan,0.002489


In [175]:
# cara 2 : one line code
df[['Regionname', 'Distance', 'Price']].groupby('Regionname').corr(method='pearson').unstack()[[('Price', 'Distance')]].apply(abs).sort_values(('Price', 'Distance')).head(1)


,Price
,Distance
Regionname,
Southern Metropolitan,0.002489


# Uji hipotesis apakah metode pembelian/Method berkaitan dengan tipe rumah/ Type (dependen) atau tidak saling berkaitan (independen)!

**Hypothesis**   
Ho : Kedua variable tidak berhubungan   
Ha : Kedua variable berhubungan  

In [183]:
df_count = pd.crosstab(df['Method'], df['Type'])
df_count

Type,h,t,u
Method,,,
PI,1069,134,361
S,6507,723,1792
SA,66,7,19
SP,1079,143,481
VB,728,107,364


In [185]:
from scipy.stats import chi2_contingency
stats, pval, dof, expected = chi2_contingency(df_count)

# conclusion
if pval <= 0.05:
    print('Tolak Ho, kedua variable berhubungan')
else:
    print('Terima Ho, kedua variable tidak berhubungan')

Tolak Ho, kedua variable berhubungan


# Uji hipotesis apakah rata-rata harga rumah (Price) antar Regionname berbeda ataukah sama! Diasumsikan persyaratan statistik parametrik terpenuhi

**Hypothesis**      
Ho : Rata rata harga rumah antar region sama      
Ha : Setidaknya ada satu pasang Regionname yang rata-rata harga
rumahnya tidak sama, tapi kita tidak tahu yang mana   

In [241]:
# get unique regionname
regionname = df['Regionname'].unique()
regionname

array(['Northern Metropolitan', 'Western Metropolitan',
       'Southern Metropolitan', 'Eastern Metropolitan',
       'South-Eastern Metropolitan', 'Eastern Victoria',
       'Northern Victoria', 'Western Victoria'], dtype=object)

In [196]:
from scipy.stats import f_oneway

In [255]:
# cara 1

stats,pval = f_oneway(
    df[df['Regionname'] == 'Northern Metropolitan']['Price'],
    df[df['Regionname'] == 'Western Metropolitan']['Price'],
    df[df['Regionname'] == 'Southern Metropolitan']['Price'],
    df[df['Regionname'] == 'Eastern Metropolitan']['Price'],
    df[df['Regionname'] == 'South-Eastern Metropolitan']['Price'],
    df[df['Regionname'] == 'Eastern Victoria']['Price'],
    df[df['Regionname'] == 'Northern Victoria']['Price'],
    df[df['Regionname'] == 'Western Victoria']['Price']
)

print(pval)

# conclusion
if pval <= 0.05:
    print('Tolak Ho')
else:
    print('Terima Ho')

0.0
Tolak Ho


In [254]:
# cara 2 : loop argument 

stats,pval = f_oneway(*[df[df['Regionname'] == i]['Price'] for i in regionname])

print(pval)

# conclusion
if pval <= 0.05:
    print('Tolak Ho')
else:
    print('Terima Ho')

0.0
Tolak Ho
